In [10]:
! pip install faiss-cpu transformers sentence-transformers

! pip install sentence_transformers

In [18]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load pre-trained models
# embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # Model for creating embeddings
# generator = pipeline('text-generation', model='gpt-3.5-turbo')  # GPT for text generation
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = HuggingFaceEmbeddings(model_name = model)
print(embeddings)

# Document to process
document = [
    "Paragraph 1: This is the first part of the document.",
    "Paragraph 2: This is the second part, which contains more information.",
    "Paragraph 3: The third paragraph has additional details.",
]

# Step 1: Create embeddings for document chunks
document_embeddings = embedding_model.encode(document)

# Step 2: Store embeddings in FAISS index
dimension = document_embeddings.shape[1]  # Get embedding dimension
index = faiss.IndexFlatL2(dimension)  # Create FAISS index
index.add(np.array(document_embeddings))  # Add embeddings to index

# Step 3: Define a function for retrieval based on a query
def retrieve_relevant_chunks(query, top_k=2):
    query_embedding = embedding_model.encode([query])  # Encode the query
    distances, indices = index.search(query_embedding, top_k)  # Search for top-k closest chunks
    return [document[i] for i in indices[0]]  # Retrieve the corresponding document chunks

# Step 4: Define the RAG function to generate response
def rag_generate(query):
    # Retrieve relevant chunks
    retrieved_chunks = retrieve_relevant_chunks(query)
    
    # Augment the query with retrieved document chunks
    augmented_query = query + "\n" + "\n".join(retrieved_chunks)
    
    # Generate the response using the generative model
    response = generator(augmented_query, max_length=100, num_return_sequences=1)
    
    return response[0]['generated_text']

# Example query
query = "What is the document about?"

# Generate response using RAG
response = rag_generate(query)
print(response)


`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


[[ 6.76568523e-02  6.34959117e-02  4.87131737e-02  7.93049857e-02
   3.74480374e-02  2.65278690e-03  3.93749885e-02 -7.09843170e-03
   5.93614578e-02  3.15370336e-02  6.00980371e-02 -5.29051535e-02
   4.06068042e-02 -2.59308834e-02  2.98428070e-02  1.12692104e-03
   7.35148638e-02 -5.03819548e-02 -1.22386672e-01  2.37029027e-02
   2.97264531e-02  4.24768589e-02  2.56337505e-02  1.99519075e-03
  -5.69191128e-02 -2.71598138e-02 -3.29036042e-02  6.60248548e-02
   1.19007125e-01 -4.58790474e-02 -7.26214498e-02 -3.25839408e-02
   5.23414090e-02  4.50553074e-02  8.25307518e-03  3.67023982e-02
  -1.39415488e-02  6.53919429e-02 -2.64272522e-02  2.06378274e-04
  -1.36643331e-02 -3.62810642e-02 -1.95043199e-02 -2.89738514e-02
   3.94270048e-02 -8.84090886e-02  2.62423395e-03  1.36713609e-02
   4.83062416e-02 -3.11565381e-02 -1.17329188e-01 -5.11690006e-02
  -8.85287821e-02 -2.18963344e-02  1.42985992e-02  4.44168001e-02
  -1.34815527e-02  7.43392482e-02  2.66382527e-02 -1.98762529e-02
   1.79191

NameError: name 'generator' is not defined

In [19]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[ 6.76568523e-02  6.34959117e-02  4.87131737e-02  7.93049857e-02
   3.74480374e-02  2.65278690e-03  3.93749885e-02 -7.09843170e-03
   5.93614578e-02  3.15370336e-02  6.00980371e-02 -5.29051535e-02
   4.06068042e-02 -2.59308834e-02  2.98428070e-02  1.12692104e-03
   7.35148638e-02 -5.03819548e-02 -1.22386672e-01  2.37029027e-02
   2.97264531e-02  4.24768589e-02  2.56337505e-02  1.99519075e-03
  -5.69191128e-02 -2.71598138e-02 -3.29036042e-02  6.60248548e-02
   1.19007125e-01 -4.58790474e-02 -7.26214498e-02 -3.25839408e-02
   5.23414090e-02  4.50553074e-02  8.25307518e-03  3.67023982e-02
  -1.39415488e-02  6.53919429e-02 -2.64272522e-02  2.06378274e-04
  -1.36643331e-02 -3.62810642e-02 -1.95043199e-02 -2.89738514e-02
   3.94270048e-02 -8.84090886e-02  2.62423395e-03  1.36713609e-02
   4.83062416e-02 -3.11565381e-02 -1.17329188e-01 -5.11690006e-02
  -8.85287821e-02 -2.18963344e-02  1.42985992e-02  4.44168001e-02
  -1.34815527e-02  7.43392482e-02  2.66382527e-02 -1.98762529e-02
   1.79191

In [20]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6528e-03,  3.9375e-02, -7.0984e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9726e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2531e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0638e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6242e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3482e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4717e